In [2]:
# импортируем библиотеки
import numpy as np 
import pandas as pd 
from itertools import product
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# загружаем данные
data_path = './'
train = pd.read_csv(data_path+'/train.csv', sep=';')
test = pd.read_csv(data_path+'/test.csv', sep=';')
test_y = pd.read_csv(data_path+'/test_answer.csv', sep=';')

In [4]:
# Дату в формат даты. В колонке Price актуальная цена.
train.Date = pd.to_datetime(train.Date, format='%d.%m.%Y')
train['Price'] = np.log(train.Regular_Price)
train.loc[train['Promo'] == 1.0,'Price'] = np.log(train.loc[train['Promo'] == 1.0,'Promo_Price']) 
# суммируем спрос по магазинам
RegionSales = train.groupby(['SKU_id','Date']).agg({'Demand':'sum','Promo':'sum','Store_id':'count','Price':'mean'}).reset_index()

In [7]:
def percentile(n):
    '''Calculate n - percentile of data'''
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'pctl%s' % n
    return percentile_

# train[['SKU_id', 'Store_id', 'Demand']].groupby(['SKU_id', 'Store_id']).agg({'mean', 'std', percentile(50)})

# добавляем лаги
def lagged_features(df 
                    , lags = [7, 14, 21, 28]
                    , windows = [7, 14]
                    , aggregation_methods = {'mean', 'median', percentile(10),  percentile(90)}
                    , promo_filters = [0, 1]
                    , deficit_filters = [0, 1]
                    , target_var = 'Demand'):

    
    filter_mapping = {0:'n', 1:'o', 2:'a'}
    
    # loop by filter variables and window
    for p, d, w in product(promo_filters, deficit_filters, windows):

        # define approproate dates for each SKU and Store pairs        
        p_idx = d_idx = df.index 
        if p <2:
            p_idx = df.Promo == p
        if d <2:
            p_idx = df.Deficit == d 

        # check whether filtered df in not empty
        if len(df[p_idx&d_idx].index)>0:
            
            # lagged features calculation
            lf_df = df[p_idx&d_idx].set_index(['SKU_id', 'Store_id', 'Date']).\
                 Demand.groupby(level=['SKU_id', 'Store_id']).apply(lambda x: x.rolling(window=w, min_periods = 1).agg(aggregation_methods))

#           # provide lags tranformations
            for l in lags:
                new_names = {x: "lag{0}_wdw{1}_{2}_{3}prm_{4}dfc".
                              format(14, 7, x, filter_mapping[1],filter_mapping[0]) for x in lf_df.columns }

                df = df.merge(lf_df.shift(l).reset_index().rename(columns = new_names),
                    how='left', on =['SKU_id', 'Store_id', 'Date'] )

    return df

In [8]:
train['Deficit'] = 0
train.Promo.fillna(0, inplace=True)

# lf = lagged_features(train, promo_filters = [1] , deficit_filters = [0])
lf = lagged_features(train[(train['SKU_id']==1) & (train['Store_id']==1)])


In [9]:
lf

,Store_id,SKU_id,Date,Promo,Demand,Regular_Price,Promo_Price,Price,Deficit,lag14_wdw7_pctl90_oprm_ndfc_x,...,lag14_wdw7_median_oprm_ndfc_y,lag14_wdw7_pctl10_oprm_ndfc_y,lag14_wdw7_pctl90_oprm_ndfc_x,lag14_wdw7_mean_oprm_ndfc_x,lag14_wdw7_median_oprm_ndfc_x,lag14_wdw7_pctl10_oprm_ndfc_x,lag14_wdw7_pctl90_oprm_ndfc_y,lag14_wdw7_mean_oprm_ndfc_y,lag14_wdw7_median_oprm_ndfc_y,lag14_wdw7_pctl10_oprm_ndfc_y
0,1,1,2015-01-01,0.0,22,163.78,NaN,5.098524,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2015-01-02,0.0,41,163.78,NaN,5.098524,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,2015-01-03,0.0,35,163.78,NaN,5.098524,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,2015-01-04,0.0,72,163.78,NaN,5.098524,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,2015-01-05,0.0,25,163.78,NaN,5.098524,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,1,1,2016-05-18,1.0,814,128.98,119.6,4.784153,0,528.8,...,170.0,87.7,563.0,275.071429,218.5,121.1,563.0,259.857143,176.0,97.0
504,1,1,2016-05-19,1.0,672,128.98,119.6,4.784153,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
505,1,1,2016-05-20,1.0,603,128.98,119.6,4.784153,0,258.4,...,170.0,87.7,563.0,257.142857,184.0,106.4,563.0,264.785714,206.0,97.0
506,1,1,2016-05-21,1.0,311,128.98,119.6,4.784153,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
